# Text Encoding

Trong quá trình xử lý dữ liệu thực tế, rất nhiều thuộc tính (cột) xuất hiện dưới dạng văn bản như tên danh mục, trạng thái, mô tả, bình luận,… Tuy nhiên, các mô hình Machine Learning không thể làm việc trực tiếp với dữ liệu dạng text. Vì vậy, bước mã hóa dữ liệu (Text Encoding) là một phần quan trọng trong giai đoạn tiền xử lý dữ liệu (data preprocessing).

Notebook này trình bày các kỹ thuật phổ biến để chuyển đổi văn bản thành dạng số, giúp mô hình có thể hiểu và học từ dữ liệu:

1. One-Hot Encoding – biến mỗi giá trị phân loại thành một vector nhị phân
2. Vectorization (TF-IDF, Bag of Words) – biểu diễn văn bản dài dưới dạng vector số

# 0. Nguồn dữ liệu

In [1]:
import pandas as pd

# --- Đường dẫn CSV ---
df = pd.read_csv('../data/interim/combined_interim.csv')

# --- Danh sách cột cần thống kê ---
fields = ['origin', 'status', 'brand', 'model', 'fuel_type', 'style']

# --- Vòng for ---
for col in fields:
    print(df[col].value_counts())

origin
Nhập khẩu             504
Lắp ráp trong nước    496
Việt Nam              424
Đang cập nhật         225
Hàn Quốc               76
Thái Lan               74
Nhật Bản               60
Nước khác              51
Mỹ                     33
Đức                    30
Trung Quốc             13
Ấn Độ                  10
Đài Loan                4
Name: count, dtype: int64
status
Đã sử dụng    971
Xe đã dùng    810
Xe mới        190
Mới            29
Name: count, dtype: int64
brand
Toyota           393
Ford             201
Hyundai          187
Kia              185
VinFast          142
Mercedes         116
Honda            101
Mitsubishi        98
Mazda             94
Lexus             63
Chevrolet         61
BMW               48
Mercedes Benz     38
MG                36
Suzuki            33
Daewoo            28
Nissan            22
Porsche           19
Volkswagen        17
Peugeot           17
Audi              17
Volvo             14
Hãng khác        12
LandRover          8
Mini          

# 1. One-hot field

In [2]:
# Chuẩn hóa và gom nhãn cho 'status'
status_mapping = {
    'đã sử dụng': 'used',
    'xe đã dùng': 'used',
    'xe mới': 'new',
    'mới': 'new'
}

df['status'] = (
    df['status']
    .astype(str)          # chuyển thành chuỗi
    .str.lower()          # lowercase để map dễ dàng
    .str.strip()          # loại bỏ khoảng trắng đầu/cuối
    .replace(status_mapping)  # gom nhãn
)
print(df['status'].value_counts())

# Chuẩn hóa và gom nhãn cho 'fuel_type'
fuel_mapping = {
    'xăng': 'gasoline',
    'dầu': 'diesel',
    'điện': 'electric',
    'hybrid': 'hybrid'
}

df['fuel_type'] = (
    df['fuel_type']
    .astype(str)
    .str.lower()
    .str.strip()
    .replace(fuel_mapping)
)

print(df['fuel_type'].value_counts())

status
used    1781
new      219
Name: count, dtype: int64
fuel_type
gasoline    1583
diesel       252
electric     106
hybrid        59
Name: count, dtype: int64


In [3]:
# One-Hot Encoding cho cả 2 cột: status và fuel_type
df = pd.get_dummies(df, columns=['status', 'fuel_type'], prefix=['status', 'fuel'])
df.to_csv('../data/interim/encoding_interim.csv', index=False, encoding='utf-8-sig')

# Kết quả
print("\n✓ Đã áp dụng One-Hot Encoding")
print(f"Số cột sau One-Hot Encoding: {df.shape[1]}")
print(f"\nCác cột mới:")
print([col for col in df.columns if col.startswith('status_') or col.startswith('fuel_')])
print("\nDữ liệu sau encoding:")
print(df.head())


✓ Đã áp dụng One-Hot Encoding
Số cột sau One-Hot Encoding: 14

Các cột mới:
['status_new', 'status_used', 'fuel_diesel', 'fuel_electric', 'fuel_gasoline', 'fuel_hybrid']

Dữ liệu sau encoding:
       price  odometer         origin      brand    model  year  \
0  149000000  159000.0      Nước khác     Toyota    Yaris  2007   
1  360000000   99000.0  Đang cập nhật      Mazda        6  2016   
2  195000000       1.0       Việt Nam     Toyota    Hiace  2008   
3  416000000   92000.0  Đang cập nhật      Honda    Civic  2019   
4  250000000   15000.0             Mỹ  Chevrolet  Orlando  2017   

           style  seats  status_new  status_used  fuel_diesel  fuel_electric  \
0            NaN    NaN       False         True        False          False   
1          Sedan    5.0       False         True        False          False   
2            Van    6.0       False         True         True          False   
3          Sedan    NaN       False         True        False          False   
4  